In [24]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import ipywidgets as widgets
import os
import urllib.request
from matplotlib import animation, rc
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from IPython.display import display, clear_output, HTML

rc('animation', html='jshtml')

# LÓGICA DEL ALGORITMO A*
def a_star(matrix, start, end):
    filas, columnas = matrix.shape
    movimientos = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: np.linalg.norm(np.array(start) - np.array(end))}

    while open_list:
        current = heapq.heappop(open_list)[1]

        if current == end:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        for dx, dy in movimientos:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < filas and 0 <= neighbor[1] < columnas:
                if matrix[neighbor[0], neighbor[1]] == 1:
                    continue

                coste_paso = np.sqrt(dx**2 + dy**2)
                tentative_g_score = g_score[current] + coste_paso

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + np.linalg.norm(np.array(neighbor) - np.array(end))
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))
    return None

# FUNCIÓN DE RENDERIZADO
def navegar_oceano(size, density, inicio_str, meta_str):
    img_file = 'barco.webp'
    if not os.path.exists(img_file):
        url = "https://raw.githubusercontent.com/antonioguismar91/AStar_Pathfinding_Python/main/barco.webp"
        try:
            urllib.request.urlretrieve(url, img_file)
        except:
            pass

    mapa = np.random.choice([0, 1], size=(size, size), p=[1-density, density])

    try:
        start = tuple(map(int, inicio_str.split(',')))
        end = tuple(map(int, meta_str.split(',')))
        mapa[start], mapa[end] = 0, 0
    except:
        print("Error: Formato de coordenadas incorrecto (ejemplo: 0,0)")
        return

    ruta = a_star(mapa, start, end)

    if not ruta:
        print("¡Ruta bloqueada! Demasiados icebergs en el camino.")
        return

    fig, ax = plt.subplots(figsize=(8, 8))

    from matplotlib.colors import ListedColormap
    cmap_custom = ListedColormap(['#0077be', '#ffffff'])
    ax.imshow(mapa, cmap=cmap_custom, zorder=1)

    # Meta
    ax.plot(end[1], end[0], 'rX', markersize=15, label="Puerto de Destino", zorder=5)

    # Trayectoria
    linea, = ax.plot([], [], color='white', linestyle='--', linewidth=2, alpha=0.6, zorder=3)

    # Imagen del barco
    try:
        img_barco = plt.imread(img_file)
        imagebox = OffsetImage(img_barco, zoom=0.07)
        ab = AnnotationBbox(imagebox, (start[1], start[0]), frameon=False, zorder=10)
        ax.add_artist(ab)
        usa_png = True
    except:
        coche_fallback, = ax.plot([], [], marker='^', color='white', markersize=12, zorder=10)
        usa_png = False

    def update(i):
        pos = ruta[i]
        if usa_png:
            ab.xybox = (pos[1], pos[0])
        else:
            coche_fallback.set_data([pos[1]], [pos[0]])

        ys, xs = zip(*ruta[:i+1])
        linea.set_data(xs, ys)

        return (ab, linea) if usa_png else (coche_fallback, linea)

    ax.set_title("⚓ NAVEGACIÓN ENTRE ICEBERGS", fontsize=14, color='white', pad=15)
    fig.patch.set_facecolor('#001a33')
    ax.axis('off')

    anim = animation.FuncAnimation(fig, update, frames=len(ruta), interval=100, blit=False)
    display(HTML(anim.to_jshtml()))
    plt.close()

# INTERFAZ DE USUARIO
s_size = widgets.IntSlider(value=20, min=10, max=30, description='Tamaño Mar')
s_dens = widgets.FloatSlider(value=0.15, min=0.05, max=0.4, step=0.05, description='Icebergs')
t_start = widgets.Text(value='0,0', description='Salida (f,c)')
t_end = widgets.Text(value='19,19', description='Meta (f,c)')
btn = widgets.Button(description="ZARPAR 🚢", button_style='info')

menu = widgets.VBox([
    widgets.HBox([s_size, s_dens]),
    widgets.HBox([t_start, t_end, btn])
])
out = widgets.Output()

def on_click(b):
    with out:
        clear_output(wait=True)
        navegar_oceano(s_size.value, s_dens.value, t_start.value, t_end.value)

btn.on_click(on_click)
display(menu, out)

Output()